## Exercise: Build a Neural Network

For this exercise, your task is to write code to build a neural network. The choice for the number of layers, and the number of neurons is up to you. However, since we will be using the MNIST dataset in this example, you will need to have 784 neurons in your input layer and 10 neurons in your output layer. Here are the steps you need to do:

1. Create your model in the `create_model()` function.
2. Return the model you created at the end of the function.

**Note**: It may take 5 - 10 minutes to download the data sets. 

In case you get stuck, you can look at the solution notebook.

### Try It Out!
- Change the number of layers and neurons in your model. How does your model accuracy change? These values are called hyperparameters and they can change the performance of our model. In a later lesson, we will learn how to automatically search for hyperparameters that give the best results.

In [ ]:
from torch import nn

def create_model():
    #TODO: Build and return a feed-forward network

In [ ]:
import numpy as np
import torch
from torchvision import datasets, transforms
from torch import nn, optim
from main import create_model

def train(model, train_loader, cost, optimizer, epoch):
    model.train()
    for e in range(epoch):
        running_loss=0
        correct=0
        for data, target in train_loader:
            data = data.view(data.shape[0], -1)
            optimizer.zero_grad()
            pred = model(data)
            loss = cost(pred, target)
            running_loss+=loss
            loss.backward()
            optimizer.step()
            pred=pred.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        print(f"Epoch {e}: Loss {running_loss/len(train_loader.dataset)}, Accuracy {100*(correct/len(train_loader.dataset))}%")

def test(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.view(data.shape[0], -1)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    print(f'Test set: Accuracy: {correct}/{len(test_loader.dataset)} = {100*(correct/len(test_loader.dataset))}%)')

training_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

testing_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

# Set Hyperparameters
batch_size=64
epoch=10

print("Downloading Data")
# Download and load the training data
trainset = datasets.MNIST('data/', download=True, train=True, transform=training_transform)
testset = datasets.MNIST('data/', download=True, train=False, transform=testing_transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

print("Loading Model")
model=create_model()

cost = nn.NLLLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

print("Starting Model Training")
train(model, train_loader, cost, optimizer, epoch)
print("Testing Trained Model")
test(model, test_loader)
